In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
tf.__version__

'2.2.0'

# 텐서플로가 제공하는 것
- 넘파이와 매우 비슷하지만 GPU를 지원
- 분산 컴퓨팅
- 일종의 JIT 컴파일러 포함. (파이썬 함수 -> 계산그래프 -> 최적화)
- 플랫폼에 중립적인 포맷 (리눅스, 파이썬 tf 모델 -> 안드로이드, 자바)
- 자동 미분, 고성능 옵티마이저
- 윈도우, 리눅스, 맥, IOS, 안드로이드 에서 작동
- C++, 자바, Go, Swift, js API

<img src='image/1.jpg' width="50%" />
텐서플로 파이썬 API

### 텐서플로 생태계
- Tensorboard
- TFX
- Tensorflow Hub

```python
t[:, 1:]
t + 10
tf.square(t)
tf.add()
tf.multiply()
tf.exp()
tf.sqrt()
```

같이 넘파이와 매우 비슷

## 하지만 다른점 !!
```python
tf.reduce_sum() <-> np.sum()
tf.reduce_mean() <-> np.mean()
```
이름이 다른 이유: tf.reduce_sum() 은 GPU 커널이 원소가 추가된 순서를 보장하지 않는 reduce 알고리즘을 사용했기 때문.

```python
tf.transpose() <-> t.T
```
텐서플로에서는 전치된 데이터의 복사본으로 새로운 텐서가 만들어짐.
넘파이에서 t.T는 동일한 데이터의 전치된 뷰 일 뿐임.

In [6]:
a = np.array([2,4,5])
t = tf.constant(a)
print(t.numpy())

print(tf.square(a))
print(np.square(t))

[2 4 5]
tf.Tensor([ 4 16 25], shape=(3,), dtype=int64)
[ 4 16 25]


## 텐서 데이터 구조
1. sparse tensor
```python
tf.SparseTensor
```
대부분이 0인 텐서를 효율적으로 나타냄. scipy.sparse_array와 비슷.
2. tensor array
```python
tf.TensorArray
```
텐서의 리스트. 리스트에 포함된 텐서는 크기와 데이터 타입이 동일 해아함.
3. ragged tensor
```python
tf.RaggedTensor
```
리스트의 리스트. 이 텐서에 포함된 값은 동일한 데이터 타입을 가져야 하지만 리스트의 길이는 달라도 됨.
4. string tensor
```python
tf.string
```
바이트 문자열을 나타내는 텐서 타입

In [7]:
# string tensor
p = tf.constant(["Café", "Coffee", "caffè", "가나다"])
r = tf.strings.unicode_decode(p, 'utf8')
r

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [44032, 45208, 45796]]>

In [8]:
# ragged tensor
r2 = tf.ragged.constant([[65, 66], [], [67]])
print(tf.concat([r, r2], axis=0))

<tf.RaggedTensor [[67, 97, 102, 233], [67, 111, 102, 102, 101, 101], [99, 97, 102, 102, 232], [44032, 45208, 45796], [65, 66], [], [67]]>


# 사용자 정의 모델, 훈련 알고리즘

### tf.keras로 가능한 것들
1. 배치 계산
2. 그래프 모드, eager 모드
3. 학습, 추론 모드
4. 마스킹
5. 상태관리 (trainable weights)
6. loss, metric 추적
7. 타입 체크(shape)
8. frozen, unfrozen
9. serialize, unserialize
10. DAG (directed acycle graph)

### 못하는 거
1. 그래디언트 계산
2. gpu 장치
3. 분산학습
4. 타입체크(데이터셋, batch 계산이 아닌것, 아웃풋이나 인풋이 없는 동작)

### 사용자 정의 Loss

In [ ]:
# 1

# loss 정의
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
# model 로드
model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

# 2

# loss 정의
class HuberLoss(keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

model.compile(loss=HuberLoss(2.), optimizer="nadam", metrics=["mae"])

# model 로드 (arg를 넘길 수 있음)
model = keras.models.load_model("my_model_with_a_custom_loss_threshold_2.h5",
                                custom_objects={"huber_fn": create_huber(2.0)})

## 다른 사용자 정의 함수들
(tf 함수를 사용!)

In [ ]:
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}
    

# model save
model = keras.models.load_model(
    "my_model_with_many_custom_parts.h5",
    custom_objects={
       "MyL1Regularizer": MyL1Regularizer,
       "my_positive_weights": my_positive_weights,
       "my_glorot_initializer": my_glorot_initializer,
       "my_softplus": my_softplus,
    })

### !! tf 2.2.0 부터는 SavedModel이 모든 케라스 레이어 지원

## 사용자 정의 지표 (metric)

keras.metric.Precision:
    정밀도를 계산하는 객체

지표는 배치마다 점진적으로 업데이트되기 때문에 streaming metric 이라고도 함.

In [9]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        #self.huber_fn = create_huber(threshold) # TODO: investigate why this fails
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def huber_fn(self, y_true, y_pred): # workaround
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [10]:
m = HuberMetric(2.)

m(tf.constant([[2.]]), tf.constant([[10.]])) 
m(tf.constant([[0.], [5.]]), tf.constant([[1.], [9.25]]))


m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=7.0>

In [11]:
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=21.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=3.0>]

In [12]:
m.reset_states()
m.variables

[<tf.Variable 'total:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'count:0' shape=() dtype=float32, numpy=0.0>]

# 사용자 정의 레이어

## 1. 가중치가 없는 레이어 (flatten, relu)
-> Lambda 사용

In [13]:
exp_layer = keras.layers.Lambda(lambda x: tf.exp(x))

## 2. 가중치가 있는 레이어
-> Layer 상속

In [ ]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

# 학습과 테스트에서 동작이 다르다면..
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

## __init__, build

케라스에서는 학습해야할 파라미터를 생성하는 것들은 build 메소드에서 하길 권장한다. 그외에는 __init__

In [ ]:
class MLPBlock(keras.Layer):
    def __init__(self):
        self.linear1 = keras.layers.Linear(32)
        self.linear2 = keras.layers.Linear(32)
    def call(self, inputs):
        x = self.linear1(inputs)
        x = tf.nn.relu(x)
        return self.linear2(x)

mlp = MLPBloc()
# --> 현재까지는 어떠한 파라미터도 생성되지 않음. (Linear() 안에 build() 가 실행되지 않았기 때문)
y = mlp(tf.ones(shape=(3, 64))) # --> MLPBlock의 call()이 실행되고 self.linear1에서 Linear()의 build가 실행되면서 파라미터 생성

# 사용자 정의 모델
-> keras.Model 상속

Model = Layer + alpha (save(), load_model(), save_weights(), compile(), fit() ....)

In [ ]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

# 모델 내부에서 로스 계산

In [ ]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        
        ###
        self.add_loss(0.05 * recon_loss)
        # metric의 경우
        self.add_metric()
        ###
        return self.out(Z)

# 자동미분

In [15]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [16]:
# tape.gradient() 가 실행된 후엔 값이 지워짐.
gradients2 = tape.gradient(z, [w1, w2])

RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [ ]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2) # works now!
del tape

In [ ]:
with tf.GradientTape(persistent=True) as hessian_tape:
    with tf.GradientTape() as jacobian_tape:
        z = f(w1, w2)
    jacobians = jacobian_tape.gradient(z, [w1, w2])
hessians = [hessian_tape.gradient(jacobian, [w1, w2])
            for jacobian in jacobians]
del hessian_tape

# 사용자 정의 학습
fit() 으로 학습이 불가능한 구조일 때

In [17]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

In [ ]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(lr=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [ ]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

# 하지만
tensorflow2.2.0 부터는 fit() 으로도 커스텀 학습 로직 작성 가능 

https://github.com/tensorflow/tensorflow/blob/master/RELEASE.md#release-220
```python
def train_step(self, data):
```
를 오버라이딩 해야함

https://github.com/tensorflow/tensorflow/blob/1381fc8e15e22402417b98e3881dfd409998daea/tensorflow/python/keras/engine/training.py#L540

왠만하면 fit()으로 해결하는 것을 추천

# 텐서플로 함수와 그래프

텐서플로 함수: 텐서인 결과를 반환하는 것

텐서플로는 텐서플로 그래프 내의 연산을 효율적으로 실행. -> 텐서플로 함수가 그냥 파이썬 함수보다 많이 빠르다!

In [ ]:
@tf.function
def tf_cube(x):
    return x ** 3

## 케라스의 사용자 정의 로스, 레이어, 모델은 텐서플로 함수로 알아서 변환 해줌.